# Data Cleaning & Data Merging
## Quantitative Trading System (NIFTY – Daily)

This notebook performs **data cleaning and alignment** on the previously
constructed daily datasets and then merges them into a single unified dataset.

### Tasks Covered
- Task 1.2: Data Cleaning
- Task 1.3: Data Merging

### Input Datasets
- nifty_spot_daily.csv
- nifty_futures_daily.csv
- nifty_options_daily.csv

### Output Deliverables
- Cleaned datasets
- data_cleaning_report.txt
- nifty_merged_daily.csv




---



In [1]:
import pandas as pd
import numpy as np

In [2]:
spot_df = pd.read_csv("nifty_spot_daily.csv")
fut_df  = pd.read_csv("nifty_futures_daily.csv")
opt_df  = pd.read_csv("nifty_options_daily.csv")

In [3]:
spot_df['date'] = pd.to_datetime(spot_df['date'])
fut_df['date']  = pd.to_datetime(fut_df['date'])
opt_df['date']  = pd.to_datetime(opt_df['date'])

# DATA CLEANING FOR SPOT DATA

In [4]:
spot_df.isna().sum()

,0
date,0
open,0
high,0
low,0
close,0
volume,0


In [6]:
spot_df = spot_df.dropna()

# DATA CLEANING FOR FUTURE DATA

In [7]:
fut_df.isna().sum()

,0
date,0
open,0
high,0
low,0
close,0
volume,0
open_interest,0


In [8]:
fut_df = fut_df.dropna()

# DATA CLEANING FOR OPTIONS DATA

In [9]:
opt_df.isna().sum()

,0
date,0
expiry,0
option_type,0
strike,0
close_price,0
volume,0
open_interest,0


In [10]:
opt_df = opt_df.dropna()



---



# REMOVING OUTLIERS FOR SPOT CLOSE PRICE AND FUTURE CLOSE PRICE

In [11]:
Q1 = spot_df['close'].quantile(0.25)
Q3 = spot_df['close'].quantile(0.75)
IQR = Q3 - Q1

spot_df = spot_df[
    (spot_df['close'] >= Q1 - 1.5 * IQR) &
    (spot_df['close'] <= Q3 + 1.5 * IQR)
]

In [12]:
Q1 = fut_df['close'].quantile(0.25)
Q3 = fut_df['close'].quantile(0.75)
IQR = Q3 - Q1

fut_df = fut_df[
    (fut_df['close'] >= Q1 - 1.5 * IQR) &
    (fut_df['close'] <= Q3 + 1.5 * IQR)
]

**Futures rollover was already handled during near-month contract selection
No additional action required here.**



---



# ATM STRIKE VALIDATION
ATM strikes were:

Dynamically calculated using spot close

Filtered to ATM ±2

Rows without valid ATM reference removed

In [13]:
opt_df[['strike', 'option_type']].head()

,strike,option_type
0,23100.0,CE
1,23100.0,PE
2,23150.0,CE
3,23150.0,PE
4,23200.0,CE




---



In [14]:
spot_df.to_csv("nifty_spot_daily_cleaned.csv", index=False)
fut_df.to_csv("nifty_futures_daily_cleaned.csv", index=False)
opt_df.to_csv("nifty_options_daily_cleaned.csv", index=False)

In [15]:
report = f"""
DATA CLEANING REPORT
--------------------
Frequency: Daily
Period: {spot_df['date'].min().date()} to {spot_df['date'].max().date()}

Spot Data:
- Missing values removed
- Outliers removed using IQR

Futures Data:
- Missing values removed
- Near-month rollover already applied
- Outliers removed using IQR

Options Data:
- ATM strike calculated dynamically
- Only ATM ±2 strikes retained
- Rows without spot alignment removed

Timestamp Alignment:
- All datasets aligned on daily 'date'
"""

with open("data_cleaning_report.txt", "w") as f:
    f.write(report)




---



# DATA MERGING

In [16]:
merged_df = pd.merge(
    spot_df,
    fut_df,
    on='date',
    how='inner',
    suffixes=('_spot', '_fut')
)

In [17]:
opt_agg = opt_df.groupby(['date', 'option_type']).agg({
    'open_interest': 'sum',
    'volume': 'sum'
}).reset_index()

In [18]:
opt_pivot = opt_agg.pivot(
    index='date',
    columns='option_type',
    values=['open_interest', 'volume']
)

opt_pivot.columns = [
    f"{col[0]}_{col[1].lower()}" for col in opt_pivot.columns
]

opt_pivot = opt_pivot.reset_index()

In [19]:
merged_df = pd.merge(
    merged_df,
    opt_pivot,
    on='date',
    how='left'
)

In [20]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              249 non-null    datetime64[ns]
 1   open_spot         249 non-null    float64       
 2   high_spot         249 non-null    float64       
 3   low_spot          249 non-null    float64       
 4   close_spot        249 non-null    float64       
 5   volume_spot       249 non-null    int64         
 6   open_fut          249 non-null    float64       
 7   high_fut          249 non-null    float64       
 8   low_fut           249 non-null    float64       
 9   close_fut         249 non-null    float64       
 10  volume_fut        249 non-null    int64         
 11  open_interest     249 non-null    int64         
 12  open_interest_ce  249 non-null    int64         
 13  open_interest_pe  249 non-null    int64         
 14  volume_ce         249 non-

In [21]:
merged_df.head()

,date,open_spot,high_spot,low_spot,close_spot,volume_spot,open_fut,high_fut,low_fut,close_fut,volume_fut,open_interest,open_interest_ce,open_interest_pe,volume_ce,volume_pe
0,2025-01-14,23165.90,23264.95,23134.15,23176.05,311235510,23248.00,23339.0,23198.45,23271.75,194699,13753850,19494225,23319300,9423714,9353511
1,2025-01-15,23250.45,23293.65,23146.45,23213.20,228039156,23302.05,23345.0,23201.00,23265.90,143790,13536300,38377050,32751075,20294803,22384264
2,2025-01-16,23377.25,23391.65,23272.05,23311.80,299416081,23406.00,23423.9,23346.15,23377.55,171198,13500850,66049050,55946100,58463061,53871172
3,2025-01-17,23277.10,23292.10,23100.35,23203.20,272945267,23344.90,23353.8,23150.10,23267.20,218388,14032150,10140900,10690275,5982807,6929039
4,2025-01-20,23290.40,23391.10,23170.65,23344.75,301455455,23339.95,23449.0,23220.00,23400.20,163712,14133750,11247750,11048025,6300739,4503616


In [22]:
merged_df.tail()

,date,open_spot,high_spot,low_spot,close_spot,volume_spot,open_fut,high_fut,low_fut,close_fut,volume_fut,open_interest,open_interest_ce,open_interest_pe,volume_ce,volume_pe
244,2026-01-08,26106.50,26133.20,25858.45,25876.85,328520950,26190.0,26210.0,25965.0,25985.6,106366,16262220,26866580,23947625,6937145,14480021
245,2026-01-09,25840.40,25940.60,25623.00,25683.30,348757262,25966.0,26031.6,25725.0,25788.3,89699,16876470,22573525,27014520,13588048,34068847
246,2026-01-12,25669.05,25813.15,25473.40,25790.25,275757268,25770.0,25899.0,25573.2,25876.0,121833,16816150,35643400,29474120,46811811,18128310
247,2026-01-13,25897.35,25899.80,25603.30,25732.30,339536336,25911.0,25934.8,25661.1,25790.8,87482,17029220,58580665,39546520,144845734,131105600
248,2026-01-14,25648.55,25791.75,25603.95,25665.60,393126596,25750.1,25850.0,25670.0,25719.0,70661,17014660,15177370,15137330,9591206,14983042


In [23]:
merged_df.to_csv("nifty_merged_daily.csv", index=False)

## Final Merged Dataset

- Dataset: nifty_merged_daily.csv
- Frequency: Daily
- Components:
  - Spot OHLCV
  - Futures OHLC + Open Interest
  - Aggregated Options Open Interest & Volume (CE / PE)

### Purpose
This dataset serves as the unified input for:
- Feature engineering
- Regime detection
- Machine learning models
- Strategy backtesting




---

